# Exploration du Master DataFrame - Données Épidémiologiques

Ce notebook explore le fichier `master_dataframe.csv` qui merge :
- **Vaccination grippe** (par département/année)
- **IAS** (Indicateur Activité Syndromique, quotidien)
- **Urgences** (passages urgences + SOS médecins, hebdomadaire)

**Objectif** : Prédire les vagues épidémiques avec un modèle d'IA

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Config visualisation
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Chargement et aperçu des données

In [ ]:
# Chargement
df = pd.read_csv('../data/processed/master_dataframe.csv')
df['date_semaine'] = pd.to_datetime(df['date_semaine'])

print(f"Shape : {df.shape}")
print(f"Période : {df['date_semaine'].min()} → {df['date_semaine'].max()}")
print(f"Régions : {df['region'].nunique()}")

In [ ]:
# Aperçu
df.head(10)

In [ ]:
# Info sur les types et valeurs manquantes
df.info()

In [ ]:
# Statistiques descriptives
df.describe()

## 2. Analyse des valeurs manquantes

In [ ]:
# Heatmap des valeurs manquantes
missing = df.isnull().sum().sort_values(ascending=False)
missing_pct = 100 * missing / len(df)

missing_df = pd.DataFrame({
    'Colonnes': missing.index,
    'Manquantes': missing.values,
    'Pourcentage': missing_pct.values
})

missing_df = missing_df[missing_df['Manquantes'] > 0]
print(missing_df.to_string(index=False))

In [ ]:
# Visualisation
plt.figure(figsize=(10, 6))
missing_df.set_index('Colonnes')['Pourcentage'].plot(kind='barh', color='coral')
plt.xlabel('% Valeurs manquantes')
plt.title('Valeurs manquantes par colonne')
plt.tight_layout()
plt.show()

## 3. Analyse temporelle

In [ ]:
# Distribution des données par région
region_counts = df['region'].value_counts().sort_values()
print(region_counts)

In [ ]:
# Distribution des données par année
year_counts = df['annee'].value_counts().sort_index()
print(year_counts)

plt.figure(figsize=(10, 5))
year_counts.plot(kind='bar', color='skyblue')
plt.xlabel('Année')
plt.ylabel('Nombre de semaines (toutes régions)')
plt.title('Distribution temporelle des données')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4. Analyse des urgences et SOS médecins

In [ ]:
# Évolution temporelle des urgences (moyenne France)
urgences_time = df.groupby('date_semaine').agg({
    'urgences_grippe': 'mean',
    'sos_medecins': 'mean'
})

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Urgences
urgences_time['urgences_grippe'].plot(ax=axes[0], color='darkred', linewidth=1.5)
axes[0].set_title('Évolution des passages aux urgences pour grippe (moyenne nationale)', fontsize=14)
axes[0].set_ylabel('Urgences grippe')
axes[0].grid(True, alpha=0.3)

# SOS Médecins
urgences_time['sos_medecins'].plot(ax=axes[1], color='darkblue', linewidth=1.5)
axes[1].set_title('Évolution des appels SOS médecins (moyenne nationale)', fontsize=14)
axes[1].set_ylabel('SOS médecins')
axes[1].set_xlabel('Date')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Analyse de l'IAS (Indicateur Activité Syndromique)

In [ ]:
# Disponibilité des données IAS
df_ias = df[df['taux_ias_moyen'].notna()]
print(f"Données IAS disponibles : {len(df_ias)} / {len(df)} lignes ({100*len(df_ias)/len(df):.1f}%)")
print(f"Période IAS : {df_ias['date_semaine'].min()} → {df_ias['date_semaine'].max()}")

In [ ]:
# Évolution IAS moyen France
if len(df_ias) > 0:
    ias_time = df_ias.groupby('date_semaine')['taux_ias_moyen'].mean()
    
    plt.figure(figsize=(14, 5))
    ias_time.plot(color='green', linewidth=2)
    plt.title('Évolution du taux IAS moyen (Indicateur Activité Syndromique)', fontsize=14)
    plt.ylabel('Taux IAS')
    plt.xlabel('Date')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 6. Comparaison régionale

In [ ]:
# Top régions avec le plus d'urgences grippe
top_regions = df.groupby('region').agg({
    'urgences_grippe': 'mean',
    'sos_medecins': 'mean'
}).sort_values('urgences_grippe', ascending=False)

print("Top 10 régions - Urgences grippe (moyenne)")
print(top_regions.head(10))

In [ ]:
# Visualisation top 5 régions
top5_regions = top_regions.head(5).index

plt.figure(figsize=(14, 6))
for region in top5_regions:
    df_region = df[df['region'] == region]
    plt.plot(df_region['date_semaine'], df_region['urgences_grippe'], 
             label=region, linewidth=1.5, alpha=0.8)

plt.title('Évolution des urgences grippe - Top 5 régions', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Urgences grippe')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Analyse de la vaccination

In [ ]:
# Données de vaccination disponibles
df_vacc = df[df['vacc_65_plus'].notna()]
print(f"Données vaccination disponibles : {len(df_vacc)} / {len(df)} lignes ({100*len(df_vacc)/len(df):.1f}%)")

if len(df_vacc) > 0:
    # Évolution moyenne par année
    vacc_year = df_vacc.groupby('annee').agg({
        'vacc_moins_65_risque': 'mean',
        'vacc_65_plus': 'mean'
    })
    
    print("\nTaux de vaccination moyen par année :")
    print(vacc_year)

In [ ]:
# Visualisation
if len(df_vacc) > 0:
    plt.figure(figsize=(12, 5))
    vacc_year.plot(kind='bar', width=0.8)
    plt.title('Évolution de la couverture vaccinale grippe par âge', fontsize=14)
    plt.ylabel('Taux de vaccination (%)')
    plt.xlabel('Année')
    plt.legend(['Moins de 65 ans à risque', '65 ans et plus'])
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 8. Corrélations entre variables

In [ ]:
# Matrice de corrélation (colonnes numériques)
numeric_cols = ['taux_ias_moyen', 'urgences_grippe', 'sos_medecins', 
                'vacc_moins_65_risque', 'vacc_65_plus', 'mois', 'semaine_annee']

corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Matrice de corrélation - Variables épidémiologiques', fontsize=14)
plt.tight_layout()
plt.show()

## 9. Analyse saisonnière

In [ ]:
# Distribution par mois
monthly_avg = df.groupby('mois').agg({
    'urgences_grippe': 'mean',
    'sos_medecins': 'mean'
})

months = ['Jan', 'Fév', 'Mar', 'Avr', 'Mai', 'Juin', 
          'Juil', 'Août', 'Sep', 'Oct', 'Nov', 'Déc']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Urgences
monthly_avg['urgences_grippe'].plot(kind='bar', ax=axes[0], color='darkred', width=0.8)
axes[0].set_title('Urgences grippe par mois (moyenne)', fontsize=12)
axes[0].set_ylabel('Urgences')
axes[0].set_xlabel('Mois')
axes[0].set_xticklabels(months, rotation=45)
axes[0].grid(True, alpha=0.3)

# SOS
monthly_avg['sos_medecins'].plot(kind='bar', ax=axes[1], color='darkblue', width=0.8)
axes[1].set_title('SOS médecins par mois (moyenne)', fontsize=12)
axes[1].set_ylabel('SOS médecins')
axes[1].set_xlabel('Mois')
axes[1].set_xticklabels(months, rotation=45)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Comparaison hiver/été
saison_avg = df.groupby('saison').agg({
    'urgences_grippe': 'mean',
    'sos_medecins': 'mean'
})

print("Moyenne par saison :")
print(saison_avg)

saison_avg.plot(kind='bar', figsize=(8, 5), width=0.7)
plt.title('Activité épidémique par saison', fontsize=14)
plt.ylabel('Moyenne')
plt.xlabel('Saison')
plt.xticks(rotation=0)
plt.legend(['Urgences grippe', 'SOS médecins'])
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Préparation pour le Machine Learning

In [ ]:
# Vérification des données exploitables pour ML
print("Préparation dataset ML :")
print("="*50)

# Dataset complet (avec urgences, target principale)
df_ml = df.dropna(subset=['urgences_grippe', 'sos_medecins'])
print(f"Lignes avec urgences/SOS : {len(df_ml)}")

# Features disponibles
features_complete = df_ml[['taux_ias_moyen', 'vacc_65_plus', 'mois', 'semaine_annee']].notna().sum()
print(f"\nDisponibilité features :")
print(features_complete)

# Target : urgences grippe
print(f"\nTarget (urgences_grippe) stats :")
print(df_ml['urgences_grippe'].describe())

In [ ]:
# Export dataset ML-ready
output_path = '../data/processed/ml_ready_dataset.csv'
df_ml.to_csv(output_path, index=False)
print(f"Dataset ML exporté : {output_path}")
print(f"Shape : {df_ml.shape}")

## Conclusions

### Points clés :

1. **Période** : Données de 2019 à 2025 (6 ans)
2. **Granularité** : Hebdomadaire × Région
3. **18 régions** françaises couvertes

### Variables principales :
- **Target** : `urgences_grippe` (passages urgences pour grippe)
- **Features** : 
  - `taux_ias_moyen` (indicateur syndromique)
  - `sos_medecins` (appels SOS)
  - `vacc_65_plus` (couverture vaccinale)
  - Features temporelles (mois, semaine, saison)

### Pour votre app :
- Utilisez `master_dataframe.csv` pour toutes les visualisations
- Filtrage facile par `region` et `date_semaine`
- Détection de pics : seuils sur `urgences_grippe` ou `taux_ias_moyen`
- Prédictions : modèle LSTM/Prophet sur série temporelle

### Prochaines étapes :
1. Feature engineering avancé (lags, rolling means)
2. Entraînement modèle de prédiction
3. Intégration dans l'app Streamlit
